In [7]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical

In [1]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical

# Constants
IMG_SIZE = 128  # Adjust if needed
data_dir = '1000images'
categories = ['Healthy', 'DiseaseRisk']  
num_classes = len(categories)

# Initialize lists to store data
data = []
labels = []

def preprocess_image(image_path):
    """Loads and processes a retinal image."""
    img = cv2.imread(image_path)
    
    # Check if the image is loaded correctly
    if img is None:
        print(f"Warning: Failed to load {image_path}. Skipping...")
        return None, None  # Return None to indicate failure

    img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))  # Resize to fixed size
    img = img / 255.0  # Normalize pixel values to [0, 1]

    return img

# Load and process images
for category in categories:
    class_num = categories.index(category)  # 0 = Healthy, 1 = DiseaseRisk
    path = os.path.join(data_dir, category)

    for img_name in os.listdir(path):
        try:
            img_path = os.path.join(path, img_name)
            img = preprocess_image(img_path)
            
            if img is not None:
                data.append(img)
                labels.append(class_num)
        except Exception as e:
            print(f"Error processing {img_name}: {e}")

# Ensure there is data to proceed
if len(data) == 0 or len(labels) == 0:
    raise ValueError("No valid images found. Please check your dataset and paths.")

# Convert lists to numpy arrays for further processing
data = np.array(data)
labels = np.array(labels)

print("Data shape:", data.shape)
print("Labels shape:", labels.shape)

# Split the dataset into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(
    data, labels, test_size=0.2, random_state=42
)

# One-hot encode the labels
y_train = to_categorical(y_train, num_classes=num_classes)
y_test = to_categorical(y_test, num_classes=num_classes)

# Build a simple CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')
])

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, epochs=50, batch_size=50, validation_split=0.20)

# Evaluate the model
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true_classes = np.argmax(y_test, axis=1)

# Confusion matrix and classification report
print("Confusion Matrix:")
print(confusion_matrix(y_true_classes, y_pred_classes))

print("\nClassification Report:")
print(classification_report(y_true_classes, y_pred_classes, target_names=categories))

# Calculate accuracy
accuracy = accuracy_score(y_true_classes, y_pred_classes)
print(f"\nModel Accuracy: {accuracy * 100:.2f}%")


Data shape: (462, 128, 128, 3)
Labels shape: (462,)


C:\Users\shrih\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 3s 255ms/step - accuracy: 0.5992 - loss: 0.6793 - val_accuracy: 0.7703 - val_loss: 0.5325
Epoch 2/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 2s 243ms/step - accuracy: 0.8022 - loss: 0.4777 - val_accuracy: 0.7703 - val_loss: 0.4922
Epoch 3/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 215ms/step - accuracy: 0.8056 - loss: 0.4305 - val_accuracy: 0.7703 - val_loss: 0.4479
Epoch 4/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 212ms/step - accuracy: 0.8041 - loss: 0.3826 - val_accuracy: 0.7973 - val_loss: 0.4317
Epoch 5/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 206ms/step - accuracy: 0.8310 - loss: 0.3614 - val_accuracy: 0.8108 - val_loss: 0.4173
Epoch 6/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 204ms/step - accuracy: 0.7965 - loss: 0.3978 - val_accuracy: 0.8243 - val_loss: 0.4283
Epoch 7/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 209ms/step - accuracy: 0.8433 - loss: 0.3657 - val_accuracy: 0.7973 - val_loss: 0.4449
Epoch 8/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 214ms/step - accuracy: 0.8108 - loss: 0.3549 - val_accuracy: 0.7973 - val_loss:

In [1]:
from tensorflow.keras.models import load_model

model = load_model('model.h5')


In [12]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model

# Constants
IMG_SIZE = 128
categories = ['Healthy', 'DiseaseRisk']  # Make sure this is defined

# Preprocess function
def preprocess_image(image_path):
    img = cv2.imread(image_path)
    if img is None:
        print(f"Failed to load {image_path}")
        return None
    img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
    img = img / 255.0
    return img
model = load_model('model.h5')

# Predict function
def predict_image(image_path):
    img = preprocess_image(image_path)
    if img is None:
        return
    img = np.expand_dims(img, axis=0)
    prediction = model.predict(img)
    predicted_class = np.argmax(prediction)
    print(f"Predicted Class: {categories[predicted_class]}")
    print(f"Confidence: {prediction[0][predicted_class] * 100:.2f}%")

# Example usage
predict_image(r"D:\sem5\1000images\Healthy\1ffa92c2-8d87-11e8-9daf-6045cb817f5b..JPG")  # <-- change path to your test image


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 370ms/step
Predicted Class: Healthy
Confidence: 100.00%
